In [3]:
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import email
import re
from datetime import datetime
from dateutil import tz
import networkx as nx
import nltk
import plotly.express as px
from plotly import graph_objects as go
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
import scipy as sp
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [4]:
df = pd.read_csv(r"emails.csv")

In [5]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [6]:
def extractmailitems(field, msg):
    elst = []
    
    for i, message in (msg.items()):
        e = email.message_from_string(message)
        elst.append(e.get(field))
       
    return elst
columns = ['Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-Folder', 'X-Origin', 'X-Filename']
for i in columns:
    df[i] = extractmailitems(i, df['message'])

def body(col):
    bodycolumn = []
    for message in col.values:
        e = email.message_from_string(message)
        bodycolumn.append(e.get_payload())
 
    return bodycolumn
df['Body'] = body(df['message'])
def employees(files):
    employees = []
    for i, employee in files.items():
        employee = employee.split('/')[0]
        employees.append(employee)
    return employees
df['Employee'] = employees(df['file'])

In [8]:

df.drop(columns = ['file', 'message'], inplace=True)
df.dropna(axis = 0, inplace=True)
df.sample(5)

,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-Filename,Body,Employee
336021,"Mon, 23 Apr 2001 22:43:00 -0700 (PDT)",pete.davis@enron.com,pete.davis@enron.com,Start Date: 4/24/01; HourAhead hour: 6; <CODE...,Schedule Crawler<pete.davis@enron.com>,pete.davis@enron.com,\steven merriss 6-28-02\Notes Folders\All docu...,MERRISS-S,steven merriss 6-28-02.nsf,Start Date: 4/24/01; HourAhead hour: 6; No an...,merriss-s
480802,"Wed, 30 Aug 2000 09:17:00 -0700 (PDT)",brent.hendry@enron.com,mark.taylor@enron.com,RE: Internet trading,Brent Hendry,Mark Taylor,\Mark_Taylor _Dec_2000\Notes Folders\Notes inbox,Taylor-M,mtaylor.nsf,I just received this today.\n\n----- Forwarded...,taylor-m
61205,"Mon, 25 Jun 2001 09:29:00 -0700 (PDT)",britt.whitman@enron.com,"john.lavorato@enron.com, louise.kitchen@enron....",California Update 6/25/01,Britt Whitman,"John J Lavorato, Louise Kitchen, David W Delainey",\Jeff_Dasovich_Oct2001\Notes Folders\All docum...,DASOVICH-J,jdasovic.nsf,EXECUTIVE SUMMARY\n? California Budget Resolve...,dasovich-j
504488,"Wed, 6 Feb 2002 12:11:44 -0800 (PST)",a..connor@enron.com,"r..brackett@enron.com, stephanie.mcginnis@enro...",LAST TEST - Please DECLINE and DELETE. Thank ...,"Connor, Richard A. </O=ENRON/OU=NA/CN=RECIPIEN...","Brackett, Debbie R. </O=ENRON/OU=NA/CN=RECIPIE...","\ExMerge - White, Stacey W.\Deleted Items",WHITE-S,stacy white 7-15-02.PST,"When: Friday, February 08, 2002 8:00 AM-8:30 A...",white-s
58941,"Thu, 31 May 2001 06:28:00 -0700 (PDT)",enron-owner@lists.qgadc.com,"enron@lists.qgadc.com, linda.robertson@enron.c...","REMINDER!! CA Energy Working Group Meeting, M...",Enron-owner@lists.qgadc.com,"enron@lists.qgadc.com, Linda.Robertson@enron.c...",\Jeff_Dasovich_June2001\Notes Folders\All docu...,DASOVICH-J,jdasovic.nsf,AT THE CALL OF LINDA ROBERTSON THERE WILL BE A...,dasovich-j


In [9]:
df['Date'] = pd.to_datetime(df['Date'], utc = True, dayfirst = True)

C:\Users\mohdt\AppData\Local\Temp\ipykernel_21348\1970594286.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], utc = True, dayfirst = True)


In [10]:
df['Year'] = pd.DatetimeIndex(df['Date']).year

In [11]:
def extract(col):
    regcol = []
    for sent in col.values:
        
        if str(sent).startswith('<'):  
            reg = re.split(r'@|\(', str(sent))[0]
            reg = re.findall(r'[a-zA-Z]+\'?-?', str(reg))           
            
        elif re.match(r'^\d+', str(sent)):
            reg = re.split(r'@|\(', str(sent))[0]
            reg = re.findall(r'[0-9]+\'?-?', str(reg))
           
        else:
            reg = re.split(r'@|<|\(', str(sent))[0]
            reg = re.findall(r'[a-zA-Z]+\'?-?', str(reg))
            
        reg = re.sub(r'[\'\",]', '', str(reg))
        regcol.append(str(reg.strip('[]')))
           
            
    return regcol
df['X-From'] = extract(df['X-From'])
df['X-To'] = extract(df['X-To'])

In [12]:
df.head()

,Date,From,To,Subject,X-From,X-To,X-Folder,X-Origin,X-Filename,Body,Employee,Year
0,2001-05-14 23:39:00+00:00,phillip.allen@enron.com,tim.belden@enron.com,,Phillip K Allen,Tim Belden,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p,2001
1,2001-05-04 20:51:00+00:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,Phillip K Allen,John J Lavorato,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,2001
2,2000-10-18 10:00:00+00:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,Phillip K Allen,Leah Van Arsdall,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,2000
3,2000-10-23 13:13:00+00:00,phillip.allen@enron.com,randall.gay@enron.com,,Phillip K Allen,Randall L Gay,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p,2000
4,2000-08-31 12:07:00+00:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,Phillip K Allen,Greg Piper,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,2000


In [22]:
def cleaningbody(col):
    msgcol = []
    for msg in col.values:
        msg = re.sub(r'[<>\n+\t+\s+\*]', ' ', msg)
        msg = re.sub(r'[0-9]+[a-zA-Z]+\d+[?!].DOC', ' ', msg)
        msg = re.sub(r'[?\s+\-+\s+?_=~]', ' ', msg)
        msg = re.sub(r' +', ' ', msg)
        msg = msg.lower().strip(' ')
        msgcol.append(msg)
    return msgcol
df['Body'] = cleaningbody(df['Body'])

In [23]:
df.to_csv('email_process.csv', index=False)

In [27]:
import pandas as pd
df = pd.read_csv(r"email_process.csv")

In [33]:
df['Subject']

0                               NaN
1                               Re:
2                          Re: test
3                               NaN
4                         Re: Hello
                    ...            
495542     Trade with John Lavorato
495543                   Gas Hedges
495544             RE: CONFIDENTIAL
495545    Calgary Analyst/Associate
495546             RE: ali's essays
Name: Subject, Length: 495547, dtype: object

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nrclex import NRCLex
import pandas as pd
sender_counts = df['From'].value_counts().reset_index()
sender_counts.columns = ['Sender', 'EmailsSent']

top_30_senders = sender_counts.head(30)

result_data = []

analyzer = SentimentIntensityAnalyzer()

for sender in top_30_senders['Sender']:
    try:
        sender_emails = df[df['From'] == sender]
        
        receiver_counts = sender_emails['To'].value_counts().reset_index()
        receiver_counts.columns = ['Receiver', 'EmailsReceived']

        top_10_receivers = receiver_counts.head(10)
        

        conversations_dict = {}
        sentiment_dict = {}
        
        for receiver in top_10_receivers['Receiver']:

            receiver_emails = sender_emails[sender_emails['To'] == receiver]
            
            conversation_text = '\n'.join(map(str, receiver_emails['Body']))
            
    
            nrc_sentiment = NRCLex(conversation_text)
            
            sentiment_scores = analyzer.polarity_scores(conversation_text)
            
            conversations_dict[receiver] = conversation_text
            sentiment_dict[receiver] = {
                'SentimentScores': sentiment_scores,
                'Emotions': nrc_sentiment.affect_frequencies
            }
        
        result_data.append({'Sender': sender, 'ReceiverList': top_10_receivers['Receiver'].tolist(), 'Conversations': conversations_dict, 'SentimentEmotion': sentiment_dict})
    
    except Exception as e:
        print(f"Error for sender '{sender}': {str(e)}")
        continue

result_df = pd.DataFrame(result_data)

result_df.to_csv('top_senders_and_data.csv', index=False)
    
    

In [26]:
selected_columns = ['Sender', 'ReceiverList', 'SentimentEmotion'] 
new_df = result_df[selected_columns]

In [28]:
new_df.to_csv('data.csv', index=False)


In [2]:
import json
import ast
import pandas as pd
df = pd.read_csv('top_senders_and_data.csv', delimiter=',')

new_data = []

for index, row in df.iterrows():
    sender = row['Sender']
    receiver_list = ast.literal_eval(row['ReceiverList'])
    conversation = ast.literal_eval(row['Conversations'])

    sender_dict = {
        'sender': sender,
        'receiverList': []
    }

    for receiver_email in receiver_list:
        if receiver_email in  conversation:
            receiver_data = {
                'receiver': receiver_email,
                ' conversation':  conversation[receiver_email]
            }
        else:
            receiver_data = receiver_email

        sender_dict['receiverList'].append(receiver_data)

    new_data.append(sender_dict)

with open('conver.json', 'w') as json_file:
    json.dump(new_data, json_file, indent=4)


In [6]:
new_df = pd.DataFrame(new_data)
new_df.to_csv('new_data.csv', index=False)